# World Happiness Report

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(highcharter)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load data

In [3]:
data <- read.csv('../input/world-happiness-report-2021/world-happiness-report.csv')
head(data)

## Exploring features

In [4]:
summary(data$Life.Ladder)

In [14]:
hchart(data$Life.Ladder, name = 'Happiness')

In [15]:
summary(data$Log.GDP.per.capita)

In [16]:
hcboxplot(x = data$Life.Ladder, var = data$Log.GDP.per.capita) %>%
    hc_chart(type = 'column') %>%
    hc_yAxis(title = list(text = 'Happiness'),
            labels = list(format = '{Life.Ladder}')) %>%
    hc_xAxis(title = list(text = 'GDP per capita'),
            labels = list(format = '{Log.GDP.per.capita}'))

In [17]:
hchart(data$Log.GDP.per.capita, name = 'GDP per capita')

## Exploring countries

In [18]:
happiest_countries <- data %>%
    select(Country.name, year, Life.Ladder) %>%
    group_by(Life.Ladder) %>%
    arrange(desc(Life.Ladder)) %>%
    head(10)
data.frame(happiest_countries)

In [20]:
value_data = data %>% 
    select(Country.name, year, Life.Ladder) %>%
    group_by(Country.name) %>%
    summarise(Count = n(), avg_happiness = mean(Life.Ladder))

value_data %>% 
    select(Country.name, avg_happiness) %>%
    arrange(desc(avg_happiness)) %>%
    head(10)

In [34]:
require(scales)
countries_plot <- value_data %>% 
    group_by(avg_happiness) %>%
    arrange(desc(avg_happiness)) %>%
    head(20) %>%
    ggplot(mapping = aes(x = Country.name, y = avg_happiness, color = Country.name, 
                        fill = Country.name, alpha = avg_happiness, size = avg_happiness)) +
    geom_bar(stat='identity') +
    coord_polar() +
    theme_minimal() +
    labs(x = 'Country', y = 'Happiness', title = 'Happiest Countries')

countries_plot

In [21]:
worldmap = map_data('world')
merged_data <- merge(x = worldmap, y = value_data, by.x = 'region', by.y = 'Country.name', all.x = TRUE) %>%
    arrange(order)
ggplot(data = merged_data, aes(x = long, y = lat, group = group)) +
    geom_polygon(aes(fill = avg_happiness)) + 
    labs(fill = 'Average Happiness')

## Relation between variables

In [22]:
linearMod <- lm(Life.Ladder ~ poly(Log.GDP.per.capita, 3, raw=TRUE), data = data)
happiness_data <- scale(data$Life.Ladder, scale = TRUE)
gdp_data <- scale(data$Log.GDP.per.capita, scale = TRUE)

ggplot(data, aes(y=Log.GDP.per.capita, x=Life.Ladder)) +
    geom_point(alpha = .5) +
    stat_smooth(method = 'lm', formula = y ~ poly(x, 3), col = 'purple') +
    theme_minimal()